In [19]:
#dict
data = open('words.txt', 'r')
words = []
for line in data:
    line = line.strip()
    words.append(line)

In [30]:
def get_matches(word):
    matches = []
    cont = 0
    for w in words:
        if w.startswith(word):
            if w != word:
                matches.append(w)
                cont += 1
        if cont == 5:
            break
    return matches

In [36]:
import tkinter as tk
import time

class AutocompleteText(tk.Text):
    def __init__(self, *args, **kwargs):
        self.callback = kwargs.pop("autocomplete", None)
        super().__init__(*args, **kwargs)
        self.bind("<Any-KeyRelease>", self._autocomplete)
        self.bind("<Tab>", self._handle_tab)

    def _handle_tab(self, event):
        tag_ranges= self.tag_ranges("autocomplete")
        if tag_ranges:
            self.mark_set("insert", tag_ranges[1])
            self.tag_remove("sel", "1.0", "end")
            self.tag_remove("autocomplete", "1.0", "end")

            return "break"

    def _autocomplete(self, event):
        if event.char and self.callback:
            word = self.get("insert-1c wordstart", "insert-1c wordend")
            inicio = time.time()
            print(inicio)
            matches = self.callback(word)
            fim = time.time()

            text_tempo["text"] = "Tempo de atualização: " + str(round((fim - inicio)*1000,2)) + " ms"

            if matches:
                remainder = matches[0][len(word):]
                insert = self.index("insert")
                self.insert(insert, remainder, ("sel", "autocomplete"))
                self.mark_set("insert", insert)
            
            texto = "Outras Sugestões: \n"
            for match in matches[1:]:
                texto += match + "\n"
            texto_outras_opcoes["text"] = texto
            

root = tk.Tk()

# titulo
root.title("Autocomplete")

# tamanho da janela
root.geometry("300x600")

# Label
label = tk.Label(root, text="Digite uma palavra: ")
label.pack(padx=10, pady=10)

# texto
text = AutocompleteText(root, autocomplete=get_matches)
text.pack(padx=10, pady=10)

# Outras opções
texto_outras_opcoes = tk.Label(root, text="Outras opções: ")
texto_outras_opcoes.pack(padx=10, pady=10)

# tempo de atualização
text_tempo = tk.Label(root, text="Tempo de atualização: ")
text_tempo.pack(padx=10, pady=10)

root.mainloop()

1697464821.7893283
